In [10]:
import random
import kfp
import kfp_server_api
import os
import string
import time
from datetime import datetime, timezone, timedelta

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# CHANGE necessary paramters here
# host is your KFP endpoint
host = 'http://127.0.0.1:8000'
num_pipelines = 100
num_pipeline_versions_per_pipeline = 100
# Use the pipeline you prefer
pipeline_file_url = 'https://storage.googleapis.com/jingzhangjz-project-pipelines/benchmarks/taxi.yaml'


def random_suffix() -> string:
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=10))


if __name__ == '__main__':
    client = kfp.Client(host)
    api_url = kfp_server_api.models.ApiUrl(pipeline_file_url)
        
    # Create pipeline latency
    create_latencies = []
    created_pipeline_ids = []
    for i in range(num_pipelines):
        api_pipeline = kfp_server_api.models.ApiPipeline(
            name='pipeline-' + random_suffix(),
            url=api_url)
        start = time.perf_counter()
        pipeline = client.pipelines.create_pipeline(body=api_pipeline)
        dur = time.perf_counter() - start
        create_latencies.append(dur)
        created_pipeline_ids.append(pipeline.id)
        
    # Create version latency    
    create_version_latencies = []
    created_version_ids = []
    for pipeline_id in created_pipeline_ids:
        for j in range(num_pipeline_versions_per_pipeline):
            key = kfp_server_api.models.ApiResourceKey(id=pipeline_id, type=kfp_server_api.models.ApiResourceType.PIPELINE)
            reference = kfp_server_api.models.ApiResourceReference(key=key, relationship=kfp_server_api.models.ApiRelationship.OWNER)
            resource_references=[reference]
            api_pipeline_version = kfp_server_api.models.ApiPipelineVersion(
                name='pipeline-version-' + random_suffix(),
                package_url=api_url,
                resource_references=resource_references)
            start = time.perf_counter()
            pipeline_version = client.pipelines.create_pipeline_version(body=api_pipeline_version)
            dur = time.perf_counter() - start
            create_version_latencies.append(dur)
            created_version_ids.append(pipeline_version.id)     
            time.sleep(2)
        
    # Get pipeline latency
    get_latencies = []
    for i in created_pipeline_ids:
        start = time.perf_counter()
        pipeline = client.pipelines.get_pipeline(i)  
        dur = time.perf_counter() - start
        get_latencies.append(dur)      
        
    # Delete pipeline latency
    delete_latencies= []
    for i in created_pipeline_ids:
        start = time.perf_counter()
        pipeline = client.pipelines.delete_pipeline(i)  
        dur = time.perf_counter() - start
        delete_latencies.append(dur)

    # Plot
    fig, axs = plt.subplots(nrows=4, figsize=(10,20))
    
    label_create_latencies = pd.Series(create_latencies, name='Create Pipeline Latency (Second)')
    sns.distplot(a=label_create_latencies, ax=axs[0])
    
    label_create_version_latencies = pd.Series(create_version_latencies, name='Create Pipeline Version Latency (Second)')
    sns.distplot(a=label_create_version_latencies, ax=axs[1])
    
    label_get_latencies = pd.Series(get_latencies, name='Get Pipeline Latency (Second)')
    sns.distplot(a=label_get_latencies, ax=axs[2])
    
    label_delete_latencies = pd.Series(delete_latencies, name='Delete Pipeline Latency (Second)')
    sns.distplot(a=label_delete_latencies, ax=axs[3])
    
    # TODO(jingzhang36): maybe dump the durations data to db or gcs, and let searborn read from there

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Trailer': 'Grpc-Trailer-Content-Type', 'Date': 'Fri, 17 Apr 2020 08:18:21 GMT', 'Transfer-Encoding': 'chunked'})
HTTP response body: {"error":"Failed to create a version.: Create pipeline version failed: Failed to get parameters from the workflow: InvalidInputError: Failed to parse the parameter.: error converting YAML to JSON: yaml: line 880: found unexpected end of stream","message":"Failed to create a version.: Create pipeline version failed: Failed to get parameters from the workflow: InvalidInputError: Failed to parse the parameter.: error converting YAML to JSON: yaml: line 880: found unexpected end of stream","code":3,"details":[{"@type":"type.googleapis.com/api.Error","error_message":"Failed to parse the parameter.","error_details":"Failed to create a version.: Create pipeline version failed: Failed to get parameters from the workflow: InvalidInputError: Failed to